# ML Project: eCommerce Inventory Prescription

## Sources:

1. Getting cass index: https://fred.stlouisfed.org/series/FRGSHPUSM649NCIS
2. Insight into potential predictive features: https://dspace.mit.edu/bitstream/handle/1721.1/126484/scm2020-huang-a-predictive-model-for-transpacific-eastbound-ocean-freight-pricing-capstone.pdf?sequence=1&isAllowed=y
3. crude oil prices: https://www.indexmundi.com/commodities/?commodity=crude-oil&months=360
4. OLD: exchange rate: https://fred.stlouisfed.org/series/EXCHUS
5. US CPI: https://fred.stlouisfed.org/series/CPIAUCSL
6. US PMI: https://data.nasdaq.com/data/ISM/MAN_PMI-pmi-composite-index
7. OLD: China PMI:


## Question for Semi: Should we change the data from 2019 to 2020? Gives us more train data - I'm doing it lol

## Data / Methodolgy

We need to predict the average US freight cost for a three month horizon. For the purposes of our problem, all items travel the same distance from the manufacturer to the warehouse. Since we are assuming that we are shipping less-than-truckload (LTL) shipments, cost is determined solely by weight of shipped items. We are using the CASS index to determine historical freight prices in the US. (insert explanation of CASS index). We can then estimate transportation costs accordingly: find average shipping cost per pound for the last month of our training set ($\text{\$/lb}_{N}$), associated with $\text{CASS Index}_{N}$, and back-compute all prior shipping costs per pound using the relationship between $\text{CASS Index}_{N}$ and the CASS index of the prior months.

To "predict" shipping rate, we will use the following features:
- oil price
- exchange rate (CH/US)   (might not need this one)
- US CPI
- US PMI
- Year
- Month

We got the above data from (sources). We found our other data (demand and individual item costs) from... (Semi)

Simple linear regression of the non-aggregated data shows that there is explanability in these features (in-sample R^2 of 0.5). 

Since we have a year of data, we can run the optimization 4 times, since our horizon is 1 quarter. Thus, we need 4 test periods, with each period correponding to 13 weeks of data. 

We'll break down the demand into 4 chunks of 13 weeks each (13 x 4 = 52). We can accordingly group the CASS data (response and features) into 3 month periods (quarters, thus 4 for each year), with the features being:
- Year
- Quarter
- Following features for the beginning of the quarter (taken from first month of quarter)
    - oil price
    - exchange rate (CH/US)   (might not need this one)
    - US CPI
    - US PMI 
    
This will let us find the neighbors of the 4 test quarters, since at the beginning of these quarters we will know the values of all the features. 

Simple linear regression of aggregated (quarterized) data leads to train R-squared of 0.5511. 

Essentially our analysis is by quarter, so we'll have projected profit (using prescription), baseline profit (simply using predicted avg CASS for quater), and ideal profit (oracle approach) for all four quarters.


Thus, work to do right now is:
- break down predicted demand into 4 matrices (19 x 13) each (we'll have to throw a week out, or include it in the last quarter or something)
- rewrite model to incorporate shipping costs per pound estimate and to account for change in horizon (change lead time to two weeks) 
- find simple prediction and oracle profits

In [159]:
using JuMP
using Gurobi
using CSV
using DataFrames
using Random, Statistics
using NearestNeighbors, Dates
ENV["COLUMNS"]=120;

#### decision vars ####
x[i,t] --> inventory order from Supplier of product i at time t

j[i,t] --> how much we are selling of product i in time t


#### technically decision vars but not really decisions ####
s[i,t] --> inventory available for sale (at AMZ warehouse) for product i at time t

m[t] --> capital (money) available to purchase inventory at time t

#### params ####
d[i,t] --> demand for product i at time t

r[i] --> sales price for product i

c[i] --> manufacturing cost for product i

tr[i] --> transportation cost for product i


we'll incorporate volume later - for initial, lets assume fixed unit costs

v[i] --> volume (size) of product i

fba --> AMZ storage fee: $19/cbm per month



revenue = 0.70 * sum( d[i,t]*r[i] for i=1:items_tot, t=1:time_tot ) --> amazon takes 30% cut

cost = sum( x[i,t]*(c[i]+tr[i]) for i=1:items_tot, t=1:time_tot ) + sum( fba*s[i,t] for i=1:items_tot, t=1:time_tot:4 )

    --> manu + transport cost of orders + monthly inventory fee for whatever inventory we have


Transportation cost is something that we want to predict -- 

w[t] = total weight of items being shipped in week t
    
$\hat{S}$[m] = uncertain shipping cost in month m - found using CASS index
    
 
 Thus, total trans cost for week t = 
 
 w[t] * S[m(t)]

## RUNNING MODEL

In [152]:
#### load data ####

# train/tests for finding KNN
trainQ1 = CSV.read("data/data_final_train_Q1.csv", DataFrame)
testQ1 = CSV.read("data/data_final_test_Q1.csv", DataFrame)
trainQ2 = CSV.read("data/data_final_train_Q2.csv", DataFrame)
testQ2 = CSV.read("data/data_final_test_Q2.csv", DataFrame)
trainQ3 = CSV.read("data/data_final_train_Q3.csv", DataFrame)
testQ3 = CSV.read("data/data_final_test_Q3.csv", DataFrame)
trainQ4 = CSV.read("data/data_final_train_Q4.csv", DataFrame)
testQ4 = CSV.read("data/data_final_test_Q4.csv", DataFrame)


# demands
demand_Q1 = CSV.read("data/salesByWeek_Q1.csv", DataFrame)
demand_Q2 = CSV.read("data/salesByWeek_Q2.csv", DataFrame)
demand_Q3 = CSV.read("data/salesByWeek_Q3.csv", DataFrame)
demand_Q4 = CSV.read("data/salesByWeek_Q4.csv", DataFrame)
d1 = Matrix(demand_Q1)[:,3:end]
d2 = Matrix(demand_Q2)[:,3:end]
d3 = Matrix(demand_Q3)[:,3:end]
d4 = Matrix(demand_Q4)[:,3:end]
Q1_startingInv = sum(d1[:,1:2], dims=2);
Q2_startingInv = sum(d2[:,1:2], dims=2);
Q3_startingInv = sum(d3[:,1:2], dims=2);
Q4_startingInv = sum(d4[:,1:2], dims=2);


# fixed item prices and fees
prices = CSV.read("data/prices.csv", DataFrame)
itemInfo = CSV.read("data/costEstimates.csv", DataFrame)

# set lead time
leadtime=2

# get constant vectors from data
r = itemInfo[:,2]
c = itemInfo[:,3]
f = itemInfo[:,5]
FBA = itemInfo[:,7]
w = itemInfo[:,10];

In [132]:
# model function
function runModel(startingCap, startingInv, nextQstartingInv, D, U)

   P = size(d1)[1]
    T = size(d1)[2]

    model = Model(with_optimizer(Gurobi.Optimizer))
    set_optimizer_attribute(model, "OutputFlag", 0)

    @variable(model, x[i=1:P, t=1:T] >= 0) # keeping it at not Int for now
    @variable(model, j[i=1:P, t=1:T] >= 0) # keeping it at not Int for now
    @variable(model, s[i=1:P, t=1:T] >= 0) # keeping it at not Int for now
    @variable(model, m[t=1:T] >= 0)
    @variable(model, lambda[t=1:T] >= 0)

    # profit (capital) in every week = last week's + 2 weeks ago's sales - this weeks cost
    @constraint(model, [t=1:T-2], m[t+2] == m[t+1] + sum(j[i,t]*(r[i] - f[i]) for i=1:P) - lambda[t+1])
    @constraint(model, m[1] == startingCap)
    @constraint(model, m[2] == m[1] - lambda[1])
    # cost in week is equal to purchasing cost + trans cost + storage cost
    @constraint(model, [t=1:T], lambda[t] == sum(x[i,t]*c[i] for i=1:P) + U*sum(x[i,t]*w[i] for i=1:P) + sum(s[i,t]*FBA[i] for i=1:P))
    # sales capped by storage and demand
    @constraint(model, [i=1:P,t=1:T], j[i,t] <= s[i,t])
    @constraint(model, [i=1:P,t=1:T], j[i,t] <= d[i,t])
    # cannot spend more than we have in capital at beginning of week
    @constraint(model, [t=1:T], lambda[t] <= m[t])
    # inventory in week is equal to prev week inv - what we sold last week + what we ordered two weeks ago
    @constraint(model, [i=1:P,t=leadtime:T-1], s[i,t+1] == s[i,t] - j[i,t] + x[i,t-1])
    @constraint(model, [i=1:P], s[i,2] == s[i,1] - j[i,1])
    @constraint(model, [i=1:P], s[i,1] == startingInv[i])
    # ending inventory must be enough to start next quarter
    @constraint(model, [i=1:P], s[i,T] - j[i,T] >= nextQstartingInv[i])
    
    @objective(model, Max, m[T] + sum(j[i,t]*(r[i] - f[i]) for i=1:P,t=T-1:T))

    optimize!(model)

    #termination_status(model)
    return objective_value(model), value.(x), value.(s), value.(j), value.(m), value.(lambda)
    
end

runModel (generic function with 3 methods)

In [157]:
function findKNN(cass_data, test_point, k)
   
    #select!(cass_data, Not(:Column1))
    append!(cass_data, test_point)

    # normalize columns (except for year and quarter)
    for col in names(cass_data)
        if col == "Year" || col == "Quarter"
            continue
        end
        cass_data[!, col] = (cass_data[!, col] .- mean(cass_data[!, col])) ./ std(cass_data[!, col])
        #test_point[!, col] = (test_point[!, col] .- mean(test_point[!, col])) ./ std(test_point[!, col])
    end
    
    test_point = DataFrame(cass_data[size(cass_data)[1], :])
    X = Array(select(cass_data[1:(size(cass_data)[1]-1), :], Not(:avgCASS)))'
    kd_tree = KDTree(X)
    test_point = Array(select(test_point, Not(:avgCASS)))'
    
    id, dist = knn(kd_tree, test_point, k)
    knn_predictions = cass_data[id[1],:].avgCASS
    return knn_predictions
end

findKNN (generic function with 1 method)

## Q1

In [160]:
# get quarter specific data
startingCap1 = 3000

U_test = findKNN(trainQ1, testQ1, 5)
U1 = 1;

# run model for q1
endQ1profit, Q1_x, Q1_s, Q1_j, Q1_m, Q1_lambda = runModel(startingCap1, Q1_startingInv, Q2_startingInv, d1, U1);

Academic license - for non-commercial use only - expires 2022-08-19


In [161]:
U_test

5-element Vector{Float64}:
  0.909220624236676
  0.9808462437285825
 -0.21074349949226198
  0.7789922402551342
  1.1208417567529192

In [134]:
endQ1profit

1.4683242606312935e6

## Q2

In [135]:
# start Q2 with profit from Q1
startingCap2 = endQ1profit

# U2 = findKNN(data_final_train_Q2, data_final_test_Q2)
U2 = 1;

# run model for q3
endQ2profit, Q2_x, Q2_s, Q2_j, Q2_m, Q2_lambda = runModel(startingCap2, Q2_startingInv, Q3_startingInv, d2, U2);

Academic license - for non-commercial use only - expires 2022-08-19


In [136]:
endQ2profit

4.265743284631293e6

## Q3

In [137]:
# start Q2 with profit from Q1
startingCap3 = endQ2profit

# get U uncertain parameters
# U3 = findKNN(data_final_train_Q3, data_final_test_Q3)
U3 = 1;

# run model for q3
endQ3profit, Q3_x, Q3_s, Q3_j, Q3_m, Q3_lambda = runModel(startingCap3, Q3_startingInv, Q4_startingInv, d3, U3);

Academic license - for non-commercial use only - expires 2022-08-19


In [138]:
endQ3profit

6.783722839631295e6

## Q4

In [140]:
# start Q2 with profit from Q1
startingCap4 = endQ3profit

# get U uncertain parameters
# U4 = findKNN(data_final_train_Q4, data_final_test_Q4)
U4 = 1;

# assuming we dont need inventory for start of 2021 - ending business at end of 2020
Q4_endingInv = zeros(size(d4)[1])

# run model for q4
endQ4profit, Q4_x, Q4_s, Q4_j, Q4_m, Q4_lambda = runModel(startingCap4, Q4_startingInv, Q4_endingInv, d4, U4);

Academic license - for non-commercial use only - expires 2022-08-19


In [141]:
endQ4profit

1.0095518856631296e7